In [1]:
#Importing all the libraries
import numpy as np
import pandas as pd
import timeit
import combine
from sklearn import svm

In [2]:
#importing the datasets
start = timeit.default_timer()
df = pd.read_csv("train.csv")
test_org = pd.read_csv("test.csv")
stop = timeit.default_timer()
print("Time taken to load data = ", stop - start)
train_org = df

Time taken to load data =  0.06713015700006508


In [3]:
#Adding exp and holiday to train
train_org = combine.comb_with_exp(train_org)
train_org = combine.comb_with_holi(train_org)

#changing the order
order = ['Year', 'Month', 'Week','Merchant_ID', 'Product_ID', 'Country', 'Expense','Exp_change', 'Holiday', 'Sales']
train_org = train_org[order]

train_org.head()

/home/somil/Codes/ZS/combine.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  expc['Exp_change'] = expc.Expense - expc.Expense.shift(1)
/home/somil/Codes/ZS/combine.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df.loc[n] = [y, mon, coun, 3, 4000,0]


,Year,Month,Week,Merchant_ID,Product_ID,Country,Expense,Exp_change,Holiday,Sales
0,2013,1,1,ar00001,1,Argentina,14749.307,0.0,1.0,157500.0
1,2013,1,1,ar00003,1,Argentina,14749.307,0.0,1.0,39375.0
2,2013,1,1,ar00004,1,Argentina,14749.307,0.0,1.0,15750.0
3,2013,1,1,ar00007,1,Argentina,14749.307,0.0,1.0,47250.0
4,2013,1,1,ar00008,1,Argentina,14749.307,0.0,1.0,283500.0


In [4]:
#dropping out S_No
test = test_org
test.drop(['S_No', 'Sales'], axis = 1, inplace = True)

#Adding exp and holiday to test
test = combine.comb_with_exp(test)
test = combine.comb_with_holi(test)

test.head()

/home/somil/Codes/ZS/combine.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  expc['Exp_change'] = expc.Expense - expc.Expense.shift(1)
/home/somil/Codes/ZS/combine.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df.loc[n] = [y, mon, coun, 3, 4000,0]


,Year,Month,Product_ID,Country,Expense,Exp_change,Holiday
0,2016,4,1,Argentina,8214.875,-2477.121,1.0
1,2016,5,1,Argentina,10777.878,2563.003,2.0
2,2016,6,1,Argentina,10320.673,-457.205,2.0
3,2016,7,1,Argentina,7377.587,-2943.086,1.0
4,2016,8,1,Argentina,9805.705,2428.118,1.0


In [5]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
sa = MinMaxScaler()

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [65]:
#Predict Function
#Inputs: Product_ID, Country, Month, Year
def Predict(traint, year, month, exp,ec, holi):
    y = np.array(traint['Sales']).reshape(traint['Sales'].shape[0],1)
    X = traint.drop('Sales', axis = 1)
    
    X = sc.fit_transform(X)
    y = sa.fit_transform(y)
    
    clf = svm.SVR(kernel='rbf', C=1e3, gamma=0.01)
    clf.fit(X, y)
    
    lin = svm.SVR(kernel = 'rbf', C=1e5, gamma=0.01)
    lin.fit(X,y)
    
    val = sc.transform([[year, month, exp, ec, holi]])
    Forecast_Sale = (sa.inverse_transform(clf.predict(val).reshape(1,1)) + sa.inverse_transform(lin.predict(val).reshape(1,1)))/2
    return Forecast_Sale

In [79]:
#Main
#Iterates through all the cases

start = timeit.default_timer()

pred = []
n = 0
for example in test.values:
    n = n + 1
    Sale = 0
    print("example :",n)
    train = train_org

    year = example[0]
    month = example[1]
    PID = example[2]
    Country = example[3]
    exp = example[4]
    ec = example[5]
    holi = example[6]
    
    train = train[train['Country'] == Country]
    train = train[train['Product_ID'] == PID]
    train.drop(['Product_ID', 'Country'], axis = 1, inplace = True)
    
    for merch in train['Merchant_ID'].unique():
        
        trainm = train[train['Merchant_ID'] == merch]
        trainm.drop('Merchant_ID', axis = 1, inplace = True)
        trainm = trainm.groupby(['Year','Month', 'Expense','Exp_change'])['Holiday', 'Sales'].apply(sum).reset_index()
        
        tf = (year - trainm.Year.values[-1])*12 + (month - trainm.Month.values[-1])
        
        Forecast_Sale = Predict(trainm, year, month, exp, ec, holi)
        Sale = Sale + Forecast_Sale
        
        #print("Merchant :", merch, "   Predicted Sale : ", Forecast_Sale)
        
    pred.append(Sale)
    
stop = timeit.default_timer()

example : 1


/home/somil/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/somil/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


example : 2
example : 3
example : 4
example : 5
example : 6
example : 7
example : 8
example : 9
example : 10
example : 11
example : 12
example : 13
example : 14
example : 15
example : 16
example : 17
example : 18
example : 19
example : 20
example : 21
example : 22
example : 23
example : 24
example : 25
example : 26
example : 27
example : 28
example : 29
example : 30
example : 31
example : 32
example : 33
example : 34
example : 35
example : 36
example : 37
example : 38
example : 39
example : 40
example : 41
example : 42
example : 43
example : 44
example : 45
example : 46
example : 47
example : 48
example : 49
example : 50
example : 51
example : 52
example : 53
example : 54
example : 55
example : 56
example : 57
example : 58
example : 59
example : 60
example : 61
example : 62
example : 63
example : 64
example : 65
example : 66
example : 67
example : 68
example : 69
example : 70
example : 71
example : 72
example : 73
example : 74
example : 75
example : 76
example : 77
example : 78
example

In [80]:
print((stop - start)/60)

24.95207097378328


In [81]:
predicted = pred

In [82]:
predicted = np.array(predicted)
predicted.shape[0]

105

In [83]:
h = []
for i in range(0,predicted.shape[0]):
    h.append(predicted[i][0][0])

In [84]:
h

[22754842.193714585,
 25592722.923672725,
 24469712.682770453,
 13188909.100734327,
 11718379.73046484,
 5583862.238771176,
 4447572.268141646,
 2843233.1799763567,
 4041383.5348501992,
 20186801.70552591,
 18659009.01575957,
 9422401.302182155,
 7615215.673229763,
 8334324.369746266,
 8390734.805658204,
 7822418.26298072,
 8685342.869447956,
 7729891.514292367,
 7947549.459924426,
 8260412.853919022,
 8377014.529101307,
 8596850.382545985,
 7935900.179814837,
 7590366.920499436,
 27656652.942977604,
 27853840.958738767,
 25746655.088842083,
 1592877.925549902,
 2134265.868522695,
 1686529.2662842853,
 1780838.4368449226,
 1812299.1403901628,
 1821798.5706194674,
 1953862.2243965713,
 2209784.9742252557,
 2161395.3292000517,
 1877847.7359977148,
 1815342.237333453,
 1832799.3280880563,
 4935118.2635361785,
 5196994.772735649,
 5255905.28446601,
 4770175.976017969,
 4371935.215127231,
 4645366.951008945,
 4404286.537123708,
 4978600.905490593,
 4802643.480882453,
 4590703.79669055,
 463

In [85]:
test_org = pd.read_csv("test.csv")
test_org['Sales'] = h

In [86]:
test_org.to_csv("submission3.csv", index = False)